In [1]:
from abi import Abi
import pandas as pd
from _const import * # paths 
from _func import * # lambdas

In [2]:
abi_obj = Abi(PATH_ABIS.ls()[0])

In [3]:
df_events = abi_obj.create_events_df()

In [4]:

df_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1 to 16
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          16 non-null     object
 1   types         16 non-null     object
 2   names         16 non-null     object
 3   indexed       16 non-null     object
 4   types_data    16 non-null     object
 5   types_topics  16 non-null     object
 6   signature     16 non-null     object
 7   hash          16 non-null     object
dtypes: object(8)
memory usage: 1.1+ KB


In [5]:
types_data_list = []
types_topics_list = []

for _, x in df_events.iterrows():
    tmp_list = list(zip(x['types'], x['indexed']))
    types_topics = []
    types_data = []
    for arg_type, indexed in tmp_list:    
        if indexed:
            types_topics.append(arg_type)
        else:
            types_data.append(arg_type)

    if not types_topics: types_topics = None
    if not types_data: types_data = None
    types_topics_list.append(types_topics)
    types_data_list.append(types_data)

df_events['types_topics'] = types_topics_list
df_events['types_data'] = types_data_list



In [6]:
df_events.head()

,name,types,names,indexed,types_data,types_topics,signature,hash
1,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
2,GuardianActivationLockChanged,"[address, address, uint256, uint256]","[guardian, lockManager, amount, total]","[True, True, False, False]","[uint256, uint256]","[address, address]","GuardianActivationLockChanged(address,address,...",0xb754cf0e7e51fa0b0a5ed45913a6da3e3aa291da6250...
3,GuardianBalanceLocked,"[address, uint256]","[guardian, amount]","[True, False]",[uint256],[address],"GuardianBalanceLocked(address,uint256)",0xc3d70b7809835f6deb56c03ebabf6aecdf78e9b4da7c...
4,GuardianBalanceUnlocked,"[address, uint256]","[guardian, amount]","[True, False]",[uint256],[address],"GuardianBalanceUnlocked(address,uint256)",0x389bab47d019f9160c099e9beca71df98fb81c079bfb...
5,GuardianDeactivationProcessed,"[address, uint64, uint256, uint64]","[guardian, availableTermId, amount, processedT...","[True, False, False, False]","[uint64, uint256, uint64]",[address],"GuardianDeactivationProcessed(address,uint64,u...",0xb7ae268c06bce2729709d55ad102f434a28f71877bfa...


In [7]:
from bq import get_logs
df_logs = get_logs(address=[ADDRESS_COURT], func_sign=[FUNC_SIGN_GUARDIANS_ACTIVATED])


            SELECT * 
            FROM `bigquery-public-data.crypto_ethereum.logs`, 
            UNNEST(topics) t 
            WHERE 
            t in (  "0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116b141b1a1a2e8f0404095"   )
            AND address in ( "0xab647b8fd9e370448d4eeb96582fe839f3d0bb24" ) 
            AND DATE(block_timestamp) >= "2020-01-01"
            LIMIT 100


In [9]:
df_logs = df_logs.merge(df_events, left_on='t', right_on='hash')

In [11]:
df_logs.head()

,log_index,transaction_hash,transaction_index,address,data,topics,block_timestamp,block_number,block_hash,t,name,types,names,indexed,types_data,types_topics,signature,hash
0,152,0x790f3476658fb72be6519a868b78e58ab99e59b0a586...,87,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-06-20 09:56:52+00:00,12670566,0x0ffd393f8b1165ec76bf3d571dc325d8766ab5108ab9...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
1,40,0x639ac978037e25f3e255451fe35cc01aec01ba152f0b...,28,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-12 12:54:00+00:00,13010465,0xea20bd418b70fb34a7ec1569ccdaf800be2430c73c00...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
2,192,0xc5a4893480835975760bd74949ec7aeae19961dfab07...,117,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-13 08:08:55+00:00,13015648,0xc9e710b874e9de3bfc37ae1ee9286f7a7d3a00da1cfe...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
3,339,0xa4272a368fe2f4a0f6beb0ef613c0ba8a77eaa6148cf...,266,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-13 08:51:56+00:00,13015853,0x8460d9d27d60b54136631366e930ab4614187fc6886e...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...
4,67,0x01a387fef0f41b9b633d94363bfa61a7fd006e01a2ae...,25,0xab647b8fd9e370448d4eeb96582fe839f3d0bb24,0x00000000000000000000000000000000000000000000...,[0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d11...,2021-08-12 04:19:17+00:00,13008145,0x774045707ca785e3017dbd030ae193b6eba8f21e0850...,0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...,GuardianActivated,"[address, uint64, uint256]","[guardian, fromTermId, amount]","[True, False, False]","[uint64, uint256]",[address],"GuardianActivated(address,uint64,uint256)",0x3bb1ff40e32fd6b2e3b81f6a7441852a5c72ec25d116...


In [25]:
def decodee(x):
    try:
        return decode_abi(x['types_data'], str.encode(x['data']))
    except Exception as e:
        return e

In [28]:
from eth_utils import decode_hex


lambda_decode_data = lambda x: decode_abi(x['types_data'], decode_hex(x['data']))
df_logs.apply(decodee, axis=1).value_counts()

Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
                                                            ..
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Padding bytes were not empty: b'0x0000000000000000000000'    1
Length: 86, dtype: int64

In [15]:
b"df_logs['data']"

TypeError: encode() missing 1 required positional argument: 'encoding'